In [1]:
import numpy as np
import random
import os
from PD.PlasmaDataset import PlasmaDataset
from PML.PlasmaModel import PlasmaModel

In [2]:
DATA_SPLIT = [0.5, 0.3, 0.2] # train/test/val splits
DATA_FRAC = 1
ORG_DATA_DIR = "./jtext_org"
HDF5_DATA_DIR = "./jtext_data/low_freq"
FEATS = ['ip', 'ip_error', 'dx', 'dy', 'AXUV_CE_66', 'AXUV_CE_70', 'AXUV_CE_74', 'AXUV_CE_78']

In [ ]:
#initialize processed data with some dummy training feature data directories for testing (uses dx and dy features)
processed_data = {
    "train_norm"   : '',
    "train_labels" : '',
    "test_norm"    : '',
    "test_labels"  : '',
    "val_norm"     : '',
    "val_labels"   : ''
}

In [3]:
def makeDataset(dataset:"PlasmaDataset", split:list, features:list, frac:float=1, preview=False):
    dataset.initialize() #creates train/test/val subdatasets
    dataset.sourceFiles(data_split = split, data_frac = frac) #initialize split/datafrac and gather hdf5 file info
    dataset.sourceData(features) #source specified feature data from files
    dataset.calcStats() #calculate data statistics from raw feature data
    dataset.normalize() #use data statistics to normalize data
    dataset.saveCSV(['train', 'test', 'val', 'stats']) #export dataset to model-loadable CSV
    if preview:
        dataset.preview() #preview datasets
    dataset.deleteDatasets() #remove dataset from memory (since saved to CSV)

In [4]:
JTEXT_LOW = PlasmaDataset(org_directory = ORG_DATA_DIR, h5_source = HDF5_DATA_DIR)
makeDataset(JTEXT_LOW, split=DATA_SPLIT, frac=DATA_FRAC, features=None)